In [ ]:
import phenopackets
from os.path import isfile
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict

In [ ]:
from phenopackets import Phenopacket
import varcode as vc
import myvariant as mv
import numpy as np
import pyensembl

In [ ]:
retinoblastoma = '../phenopackets/retinoblastoma.json'
nemalineMyopathy = '../phenopackets/nemalineMyopathy.json'
if not isfile(retinoblastoma):
    raise FileNotFoundError("Could not find phenopacket")

In [ ]:
import json

with open(nemalineMyopathy) as f:
    data = f.read()
jsondata = json.loads(data)

NMphenopacket = Parse(json.dumps(jsondata), Phenopacket())

In [ ]:
with open(retinoblastoma) as f:
    data = f.read()
jsondata = json.loads(data)

RBphenopacket = Parse(json.dumps(jsondata), Phenopacket())

In [ ]:
NMphenopacket.phenotypic_features

In [ ]:
RBphenopacket.phenotypic_features

In [ ]:
class Patient:
    def __init__(self, phenopack):
        if not isinstance(phenopack, Phenopacket):
            raise ValueError("Must be given a Phenopacket")
        self._phenopack = phenopack
        self._phenotype = self.get_hpids()
        self._genotype = self.get_variants()
    
    def get_hpids(self):
        hp_ids = [x.type for x in self._phenopack.phenotypic_features if not x.excluded]
        return hp_ids
    
    def get_variants(self):
        interp = self._phenopack.interpretations[0]
        contig = []
        start = []
        ref = []
        alt = []
        VarientInfo = mv.MyVariantInfo()

        for i in interp.diagnosis.genomic_interpretations:
            var_des = i.variant_interpretation.variation_descriptor
            newVal = VarientInfo.getvariant(var_des.id, assembly='hg38')
            contig.append(newVal[0].get('chrom'))
            ref.append(newVal[0].get('vcf').get('ref'))
            alt.append(newVal[0].get('vcf').get('alt'))
            start.append(newVal[0].get('vcf').get('position'))
        
        myVars = []
        for i in range(len(contig)):
            myVar = vc.Variant(contig[i], start[i], ref[i], alt[i], ensembl = pyensembl.ensembl_grch38)
            myVars.append(myVar)
        return myVars
    
    def get_phenopacket(self):
        return self._phenopack
    def get_phenotypes(self):
        return self._phenotype
    def get_genotypes(self):
        geno = []
        for i in self._genotype:
            geno.append(i.short_description)
        return geno
    def get_var_type(self):
        effects = []
        for i in self._genotype:
            effects.append(i.effects())
        return effects
    #def is_missense(self): TODO
        
        

In [ ]:
newPat = Patient(NMphenopacket)

In [ ]:
newPat.get_var_type()